In [1]:
import os
from dotenv import load_dotenv
import json
import time
from selenium import webdriver
from selenium.webdriver.common.by import By


In [2]:
def info_vuelos(url):

    # cargar variables de entorno
    load_dotenv()

    # opciones
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito') # entrar en ventana de incognito

    # acceder a driver de selenium
    path = os.getenv("path_chromedriver")
    driver = webdriver.Chrome(executable_path=path, options=options)

    # entrar a la pagina
    driver.get(url)
    time.sleep(10)

    # entrar al body donde esta la informacion de los vuelos
    vuelos = driver.find_elements(by=By.XPATH, value='//li[@class="body-flightsstyle__ListItemAvailableFlights-sc__sc-1p74not-5 ixybDA"]')

    # diccionario a crear
    informacion = {}
    informacion["total_vuelos"] = len(vuelos)
    informacion["info"] = []
    vuelo = 0   # enumerar los vuelos

    for i in vuelos:

        # numero de vuelo
        vuelo +=1

        # horario del vuelo
        hora = i.find_elements(by=By.XPATH, value='.//div[@class="card-flightstyle__ContainerFlightInfo-sc__sc-16r5pdw-15 cPzcrG flight-information"]/span[@class="card-flightstyle__TextHourFlight-sc__sc-16r5pdw-18 kKmcWo"]')
        hora_salida = hora[0].text
        hora_llegada = hora[1].text

        # duracion del vuelo
        duracion = i.find_element(by=By.XPATH, value='.//div[@class="card-flightstyle__ContainerFlightInfo-sc__sc-16r5pdw-15 cPzcrG flight-duration"]/span[@class="card-flightstyle__TextTotalTimeFlight-sc__sc-16r5pdw-20 fuxJTD"]')
        duracion = duracion.text

        # infomracion de escalas o si es vuelo directo
        boton_escalas = i.find_element(by=By.XPATH, value='.//div[@class="card-flightstyle__ContainerFooterCard-sc__sc-16r5pdw-24 iMBDQD"]/a')
        escala = boton_escalas.text

        # itinerario
        boton_escalas = i.find_element(by=By.XPATH, value='.//div[@class="card-flightstyle__ContainerFooterCard-sc__sc-16r5pdw-24 iMBDQD"]/a')
        boton_escalas.click()
        time.sleep(2)
        
        itinerario = i.find_element(by=By.XPATH, value='//article[@class="itinerarystyle__Wrapper-sc__sc-1n97ky6-0 hfhDJj"]/span').text

        boton_salir_escalas = driver.find_element(by=By.XPATH, value='//div[@class="MuiDialog-container MuiDialog-scrollPaper"]//span[@class="MuiButton-label"]')
        boton_salir_escalas.click()

        # acceder a la pagina de precios
        i.click()

        # extraer lista de elementos donde esta el cuadro de precios
        tarifas = i.find_elements(by=By.XPATH, value='.//ol[@class="columnBrandListstyled__BrandListDesktop-sc__sc-ghohau-1 dcOsnT"]/li[contains(@class, "columnBrandstyle__BrandWrapper")]')

        precios = []
        for j in tarifas:

            # categoria
            categoria = j.find_element(by=By.XPATH, value='./div[@class="columnBrandstyle__Brand-sc__sc-1e0tr9m-0 dPXqYU"]/div[contains(@class, "columnBrandstyle__BrandHeader")]/span').text

            # precio
            moneda = j.find_element(by=By.XPATH, value='./div[@class="columnBrandstyle__Brand-sc__sc-1e0tr9m-0 dPXqYU"]/div[contains(@class, "columnBrandstyle__BrandFooter")]//span[@class="display-currencystyle__CurrencyAmount-sc__sc-19mlo29-2 fMjBKP currency"]').text
            precio = j.find_element(by=By.XPATH, value='./div[@class="columnBrandstyle__Brand-sc__sc-1e0tr9m-0 dPXqYU"]/div[contains(@class, "columnBrandstyle__BrandFooter")]//span[@class="display-currencystyle__CurrencyAmount-sc__sc-19mlo29-2 fMjBKP"]').text
            precio = precio.replace(".", "").replace(",", "")
            precio = int(int(precio) / 100)
            
            # el aumento a la tarifa base
            aumento = j.find_element(by=By.XPATH, value='.//div[@class="differentialPricestyle__DifferentialPriceContainer-sc__sc-1wmoeew-3 kKrPZH"]//div[@class="display-currencystyle__WrapperCurrencyAmount-sc__sc-19mlo29-7 kbSkUi"]//span[@class="display-currencystyle__CurrencyAmount-sc__sc-19mlo29-2 fMjBKP"]').text
            aumento = aumento.replace(".", "").replace(",", "")     # cambiar valores para transformar a integer
            aumento = int(int(aumento) / 100)                       # quitar decimales

            # no aumenta el precio base por si mismo (solo para el caso de la categoria basic)
            if precio == aumento:
                tarifa = f"{moneda} {precio}"       # moneda COP + precio

            # suma de precio base con el precio del aumento
            else:   
                new_price = int(precio + aumento)
                tarifa = f"{moneda} {new_price}"    # moneda COP + precio con aumento
            
            # anexar informacion del segundo for a lista de precios
            precios.append({"categoria" : categoria, 
                            "precio" : tarifa
                            })

        # anexar informacion al primer for a diccionario
        informacion["info"].append({"vuelo" : vuelo, 
                                    "hora_salida" : hora_salida,
                                    "hora_llegada" : hora_llegada,
                                    "duraion_vuelo" : duracion,
                                    "escala" : escala,
                                    "itinerario" : itinerario,
                                    "tarifas" : precios
                                    })
    print("Terminado")
    return driver, informacion

Invocar funcion e imprimir

In [3]:
# url de origen y destino con fechas. Extraido de la pagina de Latam
url = "https://www.latamairlines.com/co/es/ofertas-vuelos?origin=MDE&inbound=null&outbound=2023-06-11T17%3A00%3A00.000Z&destination=CTG&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED"

driver, datos = info_vuelos(url)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_6928\2503912421.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)


Terminado


In [4]:
# Crear .json de la respuesta
with open('vuelos.json', 'w', encoding='utf-8') as archivo:
    json.dump(datos, archivo, ensure_ascii=False, indent=4)     #  ensure_ascii=False para permitir la escritura de caracteres especiales en el archivo .json
    

In [5]:
# cerrar ventana
driver.close()